In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

In [2]:
import pandas as pd

# Paths to CSV files
politifact_fake_path = './FakeNewsNet/dataset/politifact_fake.csv'
politifact_real_path = './FakeNewsNet/dataset/politifact_real.csv'
gossipcop_fake_path = './FakeNewsNet/dataset/gossipcop_fake.csv'
gossipcop_real_path = './FakeNewsNet/dataset/gossipcop_real.csv'

# Load data into DataFrames
politifact_fake = pd.read_csv(politifact_fake_path)
politifact_real = pd.read_csv(politifact_real_path)
gossipcop_fake = pd.read_csv(gossipcop_fake_path)
gossipcop_real = pd.read_csv(gossipcop_real_path)

# Preview the data
print(politifact_fake.head())
print(politifact_real.head())

                id                                           news_url  \
0  politifact15014          speedtalk.com/forum/viewtopic.php?t=51650   
1  politifact15156  politics2020.info/index.php/2018/03/13/court-o...   
2  politifact14745  www.nscdscamps.org/blog/category/parenting/467...   
3  politifact14355  https://howafrica.com/oscar-pistorius-attempts...   
4  politifact15371  http://washingtonsources.org/trump-votes-for-d...   

                                               title  \
0  BREAKING: First NFL Team Declares Bankruptcy O...   
1  Court Orders Obama To Pay $400 Million In Rest...   
2  UPDATE: Second Roy Moore Accuser Works For Mic...   
3         Oscar Pistorius Attempts To Commit Suicide   
4        Trump Votes For Death Penalty For Being Gay   

                                           tweet_ids  
0  937349434668498944\t937379378006282240\t937380...  
1  972666281441878016\t972678396575559680\t972827...  
2  929405740732870656\t929439450400264192\t929439...  
3  8

In [3]:
politifact_fake['label'] = 0
politifact_real['label'] = 1
gossipcop_fake['label'] = 0
gossipcop_real['label'] = 1

# Combine all datasets
df = pd.concat([politifact_fake, politifact_real, gossipcop_fake, gossipcop_real], ignore_index=True)

# Check combined data
print(df.shape)
print(df['label'].value_counts())

(23196, 5)
label
1    17441
0     5755
Name: count, dtype: int64


In [4]:
df.head()

,id,news_url,title,tweet_ids,label
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...,0
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...,0
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...,0
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...,0
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...,0


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X = vectorizer.fit_transform(df['title']).toarray()

import joblib
joblib.dump(vectorizer, 'vectorizer.pkl')

y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape)
print(X_train.shape, X_test.shape)

(18556, 17295) (4640, 17295)
(18556, 17295) (4640, 17295)


In [6]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [7]:
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.8398706896551724


In [8]:
joblib.dump(model, 'fake_news_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']